# 现代卷积神经网络

## AlexNet
![AlexNet](imgs/AlexNet.png)

In [4]:
import torch
from torch import nn
from d2l import torch as d2l
from torchsummary import summary

$$
\lfloor (n_h - k_h + p_h + s_h) / s_h \rfloor \times \lfloor (n_w - k_w + p_w + s_w) / s_w \rfloor
$$

$$(n_h - k_h + p_h + 1)  \times （n_w - k_w + p_w + 1）$$

In [5]:
net = nn.Sequential(
    # 输入 1 * 224 * 224
    # 注意下面计算shape时 padding * 2

    # floor 224 - 11 + 1*2 + 4 / 4 =  [224-11+6]/4 = 219 / 4 = 54
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), nn.ReLU(),

    # 54 - 3 + 2 / 2 = 53 / 2 = 26
    nn.MaxPool2d(kernel_size=3, stride=2),

    # 26 - 5 + 2*2 + 1 = 26
    nn.Conv2d(96, 256, kernel_size=5, padding=2), nn.ReLU(),

    # 26 - 3 + 2 / 2 = 12
    nn.MaxPool2d(kernel_size=3, stride=2),

    # 12 - 3 + 1 * 2 + 1 = 12
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(),

    # 12 - 3 + 1 * 2 + 1 = 12
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),

    # 12 - 3 + 1 * 2 + 1 = 12
    nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),

    # （12 - 3 + 2） / 2 = 5
    nn.MaxPool2d(kernel_size=3, stride=2),

    # 256 * 5 * 5 = 4600
    nn.Flatten(),
    # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合
    nn.Linear(6400, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    # 最后是输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
    nn.Linear(4096, 10))

In [6]:
summary(net, input_size=(1,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          11,712
              ReLU-2           [-1, 96, 54, 54]               0
         MaxPool2d-3           [-1, 96, 26, 26]               0
            Conv2d-4          [-1, 256, 26, 26]         614,656
              ReLU-5          [-1, 256, 26, 26]               0
         MaxPool2d-6          [-1, 256, 12, 12]               0
            Conv2d-7          [-1, 384, 12, 12]         885,120
              ReLU-8          [-1, 384, 12, 12]               0
            Conv2d-9          [-1, 384, 12, 12]       1,327,488
             ReLU-10          [-1, 384, 12, 12]               0
           Conv2d-11          [-1, 256, 12, 12]         884,992
             ReLU-12          [-1, 256, 12, 12]               0
        MaxPool2d-13            [-1, 256, 5, 5]               0
          Flatten-14                 [-

In [7]:
X = torch.randn(1, 1, 224, 224)
for layer in net:
    X=layer(X)
    print(layer.__class__.__name__,'output shape:\t',X.shape)

Conv2d output shape:	 torch.Size([1, 96, 54, 54])
ReLU output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Conv2d output shape:	 torch.Size([1, 256, 26, 26])
ReLU output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Conv2d output shape:	 torch.Size([1, 384, 12, 12])
ReLU output shape:	 torch.Size([1, 384, 12, 12])
Conv2d output shape:	 torch.Size([1, 384, 12, 12])
ReLU output shape:	 torch.Size([1, 384, 12, 12])
Conv2d output shape:	 torch.Size([1, 256, 12, 12])
ReLU output shape:	 torch.Size([1, 256, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 256, 5, 5])
Flatten output shape:	 torch.Size([1, 6400])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1,

In [8]:
batch_size = 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)

In [9]:
# lr, num_epochs = 0.01, 1
# d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

## VGG
![VGG](imgs/VGG.png)

In [10]:
import torch
from torch import nn
from d2l import torch as d2l
def vgg_block(num_convs, in_channels, out_channels):
    '''

    :param num_convs: 卷积层的数量
    :param in_channels: 输入通道的数量
    :param out_channels: 输出通道的数量
    :return: layers
    '''
    layers = []
    for num in range(num_convs):
        layers.append(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        )
        layers.append(
            nn.ReLU()
        )
        in_channels = out_channels
    layers.append(
        nn.MaxPool2d(2,stride=2)
    )
    return nn.Sequential(*layers)


上面的代码实现了VGG块，
其中有超参数变量conv_arch指定了每个VGG块里卷积层的个数，和输出通道数

原始的VGG共有5个卷积块，
其中前两个块各有一个卷积层，
后三个块各包含2个卷积层。

第一个模块有64个输出通道，后续输出通道把输出通道数量翻倍。直到512.

由于该网络使用
2 * 1 + 3 * 2 = 8个卷积层
和3个全连接层，
因此它通常被称为VGG-11。

In [11]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2,512), (2, 512))

def vgg(conv_arch):
    conv_blk = []
    in_channels = 1

    # 卷积层部分
    for (num_convs, out_channels) in conv_arch:
        conv_blk.append(
            vgg_block(num_convs, in_channels, out_channels)
        )
        in_channels = out_channels
    return nn.Sequential(
        *conv_blk,
        nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10)
    )
net = vgg(conv_arch)

X = torch.randn(size=(1,1,224,224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, 'output shape: ', X.shape)

Sequential output shape:  torch.Size([1, 64, 112, 112])
Sequential output shape:  torch.Size([1, 128, 56, 56])
Sequential output shape:  torch.Size([1, 256, 28, 28])
Sequential output shape:  torch.Size([1, 512, 14, 14])
Sequential output shape:  torch.Size([1, 512, 7, 7])
Flatten output shape:  torch.Size([1, 25088])
Linear output shape:  torch.Size([1, 4096])
ReLU output shape:  torch.Size([1, 4096])
Dropout output shape:  torch.Size([1, 4096])
Linear output shape:  torch.Size([1, 4096])
ReLU output shape:  torch.Size([1, 4096])
Dropout output shape:  torch.Size([1, 4096])
Linear output shape:  torch.Size([1, 10])


由于VGG-11比AlexNet计算量更大， 因此我们构建一个通道较少的网络， 足够用来训练Fashion-MNIST数据集

In [12]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

# lr, num_epochs, batch_size = 0.05, 1, 128
# train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
# d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

## 网络中的网络NiN

AlexNet，VGG等都有一个共同的设计模式：

通过一系列的卷积和汇聚层来提取空间结构特征；
然后通过全连接层对特征的表征进行处理。

它们的最大改进在于如何扩大和加深这两个模块。

或者，可以想象在这个过程的早期使用全连接层。
然而，使用了全连接层，可能会完全放弃表征的空间结构。

网络中的网络（NiN）提供了一种非常简单的解决方案：

**在每个像素的通道上分别使用多层感知机**

### NiN块
回想一下，卷积层的输入和输出由4维张量组成，
张量的每个轴分别对应样本、通道、高度和宽度。

NiN的想法是在每个像素位置（针对每个高度和宽度）应用一个全连接层。 如果我们将权重连接到每个空间位置，我们可以将其视为卷积层

![NiN](./imgs/NiN.png)

In [13]:
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels,out_channels,kernel_size,strides,padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU()
    )

net = nn.Sequential(
    # 输入
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d((1, 1)),
    # 将四维的输出转成二维的输出，其形状为(批量大小,10)
    nn.Flatten())

In [14]:
# lr, num_epochs, batch_size = 0.1, 1, 128
# train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
# d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

In [15]:
summary(net, input_size=(1,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          11,712
              ReLU-2           [-1, 96, 54, 54]               0
            Conv2d-3           [-1, 96, 54, 54]           9,312
              ReLU-4           [-1, 96, 54, 54]               0
            Conv2d-5           [-1, 96, 54, 54]           9,312
              ReLU-6           [-1, 96, 54, 54]               0
         MaxPool2d-7           [-1, 96, 26, 26]               0
            Conv2d-8          [-1, 256, 26, 26]         614,656
              ReLU-9          [-1, 256, 26, 26]               0
           Conv2d-10          [-1, 256, 26, 26]          65,792
             ReLU-11          [-1, 256, 26, 26]               0
           Conv2d-12          [-1, 256, 26, 26]          65,792
             ReLU-13          [-1, 256, 26, 26]               0
        MaxPool2d-14          [-1, 256,

# GoogLeNet
GoogLeNet吸收了NiN中串联网络的思想，并在此基础上做了改进。
这篇论文的一个重点是解决了什么样大小的卷积核最合适的问题。

这篇论文的一个观点是，有时使用不同大小的卷积核是有利的。

## Inception块
![inception](imgs/inceptionblk.png)

如图，Inception块由4条

**并行路径**
组成。

这4条路径都使用合适的填充来使输入和输出的高宽一致。
最后，我们把每条线路的输出在通道维度上连结，
构成Inception的输出。

在Inception块中，通常调整的超参数是每层输出的通道数。

In [16]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

class Inception(nn.Module):
    '''c1-c4是每条路径的输出通道数'''
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__()
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)
    def forward(self,x):
        # 并联的含义
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 并联，0为批次，1为通道，2为H，3为W
        return torch.cat((p1,p2,p3,p4),dim=1)

GoogLeNet共使用9个Inception块和全局平均汇聚层的堆叠来生成其估计值。

Inception块之间的最大汇聚层可以降低维度。

第一个模块类似于AlexNet和LeNet，
Inception块的组合从VGG继承，

**全局平均汇聚层避免了在最后使用全连接层**

# 补充-全局平均汇聚层避免了在最后使用全连接层

+ 全局平均池化代替全连接层虽然可以减少模型的参数量，防止模型发生过拟合，但不利于模型的迁移学习，
+ 而全连接层则可以更好的进行迁移学习，因为它的参数调整很大一部分是在全连接层中，迁移的时候虽然卷积层的参数也会调整，但是相对来说要小很多

参考链接
+ [为什么为什么全局平均池化层有用，为什么可以替代全连接层？ - 刘冬煜的回答 - 知乎](https://www.zhihu.com/question/373188099/answer/1026457443)
+ [全局平均池化代替全连接层，全连接层的作用？](https://blog.csdn.net/m0_45388819/article/details/120683865)

## gooLeNet
![googLent](imgs/googLeNet.png)

如图，gooLeNet一共使用个Inception块和GAP的堆叠来生成其估计值

$$
\lfloor (n_h - k_h + p_h + s_h) / s_h \rfloor \times \lfloor (n_w - k_w + p_w + s_w) / s_w \rfloor
$$

$$(n_h - k_h + p_h + 1)  \times （n_w - k_w + p_w + 1）$$

In [17]:
b1 = nn.Sequential(
    # 输入 1 * 96 * 96

    # floor（96 - 7 + 3*2 + 2） / 2 = 97 / 2 = 48
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.ReLU(),

    # # 48 - 3 + 1*2 + 1 = 48
    # nn.Conv2d(64, 192, kernel_size=3, padding=1),
    # nn.ReLU(),

    # ( 48 - 3 + 1 * 2 + 2 ) / 2 = 24
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)
# 输出 64 * 24 * 24

In [18]:
summary(b1, input_size=(1,96,96))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 48, 48]           3,200
              ReLU-2           [-1, 64, 48, 48]               0
         MaxPool2d-3           [-1, 64, 24, 24]               0
Total params: 3,200
Trainable params: 3,200
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.04
Forward/backward pass size (MB): 2.53
Params size (MB): 0.01
Estimated Total Size (MB): 2.58
----------------------------------------------------------------


第二个模块使用两个卷积层：
+ 第一个卷积层是64个通道、1 * 1卷积
+ 第二个卷积层使用将通道数量乘以3倍的3*3卷积层

这对应Inception块中的第二条路径

In [19]:
b2 = nn.Sequential(
    # 输入 64 * 24 * 24

    # 24 - 1 + 1 = 24
    nn.Conv2d(64, 64, kernel_size=1),
    nn.ReLU(),

    # 24 - 3 + 1 + 1 * 2 = 24
    nn.Conv2d(64, 192,kernel_size=3, padding=1),
    nn.ReLU(),

    # (24 - 3 + 1 * 2 + 2) / 2 = 12
    nn.MaxPool2d(kernel_size=3,padding=1, stride=2)
)
# 输出 192 * 12 * 12

In [20]:
summary(b2, input_size=(64, 24, 24))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 24, 24]           4,160
              ReLU-2           [-1, 64, 24, 24]               0
            Conv2d-3          [-1, 192, 24, 24]         110,784
              ReLU-4          [-1, 192, 24, 24]               0
         MaxPool2d-5          [-1, 192, 12, 12]               0
Total params: 114,944
Trainable params: 114,944
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.14
Forward/backward pass size (MB): 2.46
Params size (MB): 0.44
Estimated Total Size (MB): 3.04
----------------------------------------------------------------


In [21]:
b3 = nn.Sequential(
    # 输入 192 * 12 * 12

    # in_channel是输入通道，c1-c4是每条路径的输出通道数（输入通道数是一样的）

    # 输出通道数 = 64 + 128 + 32 + 32 = 256
    # Inception block不改变特征图的尺寸
    Inception(192, 64, (96, 128), (16, 32), 32),

    # 输出通道数 = 128 + 192 + 96 + 64 = 480
    Inception(256, 128, (128, 192), (32, 96), 64),

    # 改变尺寸一半，输出通道数不变
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)
# 输出 480 * 6 * 6

In [22]:
summary(b3, input_size=(192, 12, 12))
# 观察Inception block不会改变特征图的尺寸

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 12, 12]          12,352
            Conv2d-2           [-1, 96, 12, 12]          18,528
            Conv2d-3          [-1, 128, 12, 12]         110,720
            Conv2d-4           [-1, 16, 12, 12]           3,088
            Conv2d-5           [-1, 32, 12, 12]          12,832
         MaxPool2d-6          [-1, 192, 12, 12]               0
            Conv2d-7           [-1, 32, 12, 12]           6,176
         Inception-8          [-1, 256, 12, 12]               0
            Conv2d-9          [-1, 128, 12, 12]          32,896
           Conv2d-10          [-1, 128, 12, 12]          32,896
           Conv2d-11          [-1, 192, 12, 12]         221,376
           Conv2d-12           [-1, 32, 12, 12]           8,224
           Conv2d-13           [-1, 96, 12, 12]          76,896
        MaxPool2d-14          [-1, 256,

In [23]:
b4 = nn.Sequential(
    # 输入 480 * 6 * 6

    Inception(480, 192, (96, 208), (16, 48), 64),

    Inception(512, 160, (112, 224), (24, 64), 64),

    Inception(512, 128, (128, 256), (24, 64), 64),

    Inception(512, 112, (144, 288), (32, 64), 64),

    Inception(528, 256, (160, 320), (32, 128), 128),

    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

# 输出 832 * 3 * 3

In [24]:
summary(b4, input_size=(480,6,6))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 192, 6, 6]          92,352
            Conv2d-2             [-1, 96, 6, 6]          46,176
            Conv2d-3            [-1, 208, 6, 6]         179,920
            Conv2d-4             [-1, 16, 6, 6]           7,696
            Conv2d-5             [-1, 48, 6, 6]          19,248
         MaxPool2d-6            [-1, 480, 6, 6]               0
            Conv2d-7             [-1, 64, 6, 6]          30,784
         Inception-8            [-1, 512, 6, 6]               0
            Conv2d-9            [-1, 160, 6, 6]          82,080
           Conv2d-10            [-1, 112, 6, 6]          57,456
           Conv2d-11            [-1, 224, 6, 6]         226,016
           Conv2d-12             [-1, 24, 6, 6]          12,312
           Conv2d-13             [-1, 64, 6, 6]          38,464
        MaxPool2d-14            [-1, 51

In [25]:
b5 = nn.Sequential(
    # 输入832 * 3 * 3

    Inception(832, 256, (160, 320), (32, 128), 128),
    Inception(832, 384, (192, 384), (48, 128), 128),

    # 输出 1024 * 3 * 3

    nn.AdaptiveAvgPool2d((1,1)),
    # 池化后的每个通道上的大小是一个1x1的,
    # 也就是每个通道上只有一个像素点. (1, 1)表示的outputsize
    nn.Flatten()
)

In [26]:
summary(b5, input_size=(832, 3,3))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 256, 3, 3]         213,248
            Conv2d-2            [-1, 160, 3, 3]         133,280
            Conv2d-3            [-1, 320, 3, 3]         461,120
            Conv2d-4             [-1, 32, 3, 3]          26,656
            Conv2d-5            [-1, 128, 3, 3]         102,528
         MaxPool2d-6            [-1, 832, 3, 3]               0
            Conv2d-7            [-1, 128, 3, 3]         106,624
         Inception-8            [-1, 832, 3, 3]               0
            Conv2d-9            [-1, 384, 3, 3]         319,872
           Conv2d-10            [-1, 192, 3, 3]         159,936
           Conv2d-11            [-1, 384, 3, 3]         663,936
           Conv2d-12             [-1, 48, 3, 3]          39,984
           Conv2d-13            [-1, 128, 3, 3]         153,728
        MaxPool2d-14            [-1, 83

In [27]:
net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 10))

In [28]:
summary(net, input_size=(1,96,96))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 48, 48]           3,200
              ReLU-2           [-1, 64, 48, 48]               0
         MaxPool2d-3           [-1, 64, 24, 24]               0
            Conv2d-4           [-1, 64, 24, 24]           4,160
              ReLU-5           [-1, 64, 24, 24]               0
            Conv2d-6          [-1, 192, 24, 24]         110,784
              ReLU-7          [-1, 192, 24, 24]               0
         MaxPool2d-8          [-1, 192, 12, 12]               0
            Conv2d-9           [-1, 64, 12, 12]          12,352
           Conv2d-10           [-1, 96, 12, 12]          18,528
           Conv2d-11          [-1, 128, 12, 12]         110,720
           Conv2d-12           [-1, 16, 12, 12]           3,088
           Conv2d-13           [-1, 32, 12, 12]          12,832
        MaxPool2d-14          [-1, 192,

In [29]:
# lr, num_epochs, batch_size = 0.1, 1, 128
# train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
# d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

# 批量规范化

训练深层神经网络是十分困难的，特别是在较短的时间内使他们收敛更加棘手。


## 为什么需要批量规范化层

+ 首先，数据预处理的方式通常会对最终结果产生巨大影响。

    我们应用多层感知机来预测房价的例子时，我们的第一步是标准化输入特征，使其平均值为0，方差为1。
    直观地说，这种标准化可以很好地与我们的优化器配合使用，因为它可以将参数的量级进行统一。



+ 第二，对于典型的多层感知机或卷积神经网络训练时，中间层中的变量可能具有更广的变化范围：

    不论是沿着从输入到输出的层，跨同一层中的单元，或是随着时间的推移，模型参数的随着训练更新变幻莫测。
    直观地说，我们可能会猜想，如果一个层的可变值是另一层的100倍，这可能需要对学习率进行补偿调整。


+ 第三，更深层的网络很复杂，容易过拟合。 这意味着正则化变得更加重要


其原理如下：
在每次训练迭代中，我们首先规范化输入，

即通过减去其均值并除以其标准差，其中两者均基于当前小批量处理。

接下来，我们应用比例系数和比例偏移。

正是由于这个基于批量统计的标准化，
才有了批量规范化的名称。

## 请注意

1. 如果我们尝试使用大小为1的小批量应用批量规范化，我们将无法学到任何东西。
这是因为在减去均值之后，每个隐藏单元将为0。
所以，只有使用足够大的小批量，批量规范化这种方法才是有效且稳定的。

2. **批量大小的选择可能比没有批量规范化时更重要**
也就是说，宁愿不用BN，也不要用很小的batch_size。

3. 我们在方差估计值中添加一个小的常量ε > 0，以防止除以0

4. 批量规范化层在“训练模式”（通过小批量统计数据规范化）和预测模式（通过数据集统计规范化）中的功能不同。
在训练过程中，我们无法得知使用整个数据集来估计平均值和方差，所以只能根据每个小批次的平均值和方差不断训练模型。
而在预测模式下，可以根据整个数据集精确计算批量规范化所需的平均值和方差

回想一下，批量规范化层和其他层的一个关键区别是，
由于批量规范化在完整的小批量上运行，

因此我们不能像以前在引入其他层时那样忽略批量大小。

我们在下面讨论这两种情况：全连接层和卷积层，他们的批量规范化实现略有不同。

## 全连接层


通常，我们把批量规范化层置于

**全连接层的仿射变换和激活函数之间**。

设全连接层的输入为x，权重参数与偏置参数分别的W和b，激活函数为Φ，批量归一化符号为BN。
则，使用批量归一化的全连接层的输出计算为：
$$
h = \phi(BN(Wx+b))
$$

回想一下，均值和方差是在应用变换的“相同”小批量上计算的。

## 卷积层

同样，对于卷积层，我们可以在卷积层之后和非线性激活函数之前应用批量规范化。

当卷积有多个输出通道时，

我们需要对这些通道的“每个”输出执行批量规范化，

每个通道都有自己的拉伸（scale）和偏移（shift）参数，
这两个参数都是标量。

假设我们的小批量包含m个样本，并且对于每个通道，卷积的输出具有高度p和宽度q。

那么对于卷积层，

**我们在每个输出通道的m\*p\*q元素上同时执行每个批量标准化**

因此，在计算mean和std时，我们会所有空间位置的值，然后在给定通道内应用相同的均值和方差，
以便在每个空间位置对值进行规范化。

但是，
正如我们前面提到的，批量规范化在训练模式和预测模式下的行为通常不同。

首先，将训练好的模型用于预测时，我们不再需要样本均值中的噪声以及在微批次上估计每个小批次产生的样本方差了。
其次，例如，我们可能需要使用我们的模型对逐个样本进行预测。

一种常用的方法是通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。
可见，和暂退法一样，批量规范化层在训练模式和预测模式下的计算结果也是不一样的。

In [30]:
def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    '''

    :param X:
    :param gamma:
    :param beta:
    :param moving_mean:
    :param moving_var:
    :param eps:
    :param momentum:
    :return:
    '''
    # 判断当前模式时训练模式还是预测模式
    if not torch.is_grad_enabled():
        # 预测推理模式，直接使用传入的移动均值和移动方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        # 训练模式
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 全连接层的情况，计算特征维上的均值和方差（竖方向的）
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积的情况，计算通道维度上的（axis=1）均值与方差（B，C，H，W）
            # 这里我们要保持X的形状以便后面广播运算
            mean = X.mean(dim=(0, 2, 3),keepdim=True)
            var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)
        # 训练模式下用当前的mean和var做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动的均值和方差
        moving_mean = momentum  * moving_mean + (1. - momentum) * mean
        moving_var = momentum * moving_var + (1. - momentum) * var
    Y = gamma * X_hat + beta # 缩放和移位
    return Y, moving_mean.data, moving_var.data

In [31]:
# 模拟全连接层
X = torch.Tensor(
    [[1,2],
     [3,4]]
)

X.mean(dim=0, keepdim=True), len(X)
# 相当于表格对列求均值

(tensor([[2., 3.]]), 2)

In [36]:
# 测试卷积层
X = torch.Tensor([
    [ # 批次
        [[1,2],[3,4]], # 通道1
        [[5,6],[7,8]], # 通道2
        [[9,10],[11,12]] # 通道3
    ],
    [
        [[1,2],[3,4]], #通道1
        [[5,6],[7,8]], # 通道2
        [[9,10],[11,12]] # 通道3
    ]
])
X.shape

torch.Size([2, 3, 2, 2])

In [38]:
X.mean(dim=(0,2,3),keepdim=True) # 相当于每一个通道求均值
# 1 + 2 + 3 + 4 + 1 + 2 + 3 + 4 / 8 = 2.5
# （% + 6 + 7 + 8 ） * 2 / 8
# 相当于一摞扑克牌，（4）个花色为一个样本，求某一个花色的均值

tensor([[[[ 2.5000]],

         [[ 6.5000]],

         [[10.5000]]]])

现在我们可以创建一个正确的BatchNorm层，
这个层将保持适当的参数：拉伸参数gamma和偏移beta。

这两个参数将在训练中更新。
此外，我们的层将保存均值和方差的移动平均值，以便在模型预测期间随后使用。



抛开算法细节，注意我们实现层的设计模式。

通常情况，我们用一个单独的函数定义其数学原理，比如batch_norm，然后再将功能集成到一个自定义层中，

其代码主要处理数据转移到训练设备，分配和初始化任何必须的变量，跟踪移动均值。

In [74]:
class BatchNorm(nn.Module):

    def __init__(self, num_features, num_dims):
        """

        :param num_features: 完全连接层的输出数量或卷积层的输出通道数。
        :param num_dims: 2表示完全连接层，4表示卷积层
        """
        super().__init__()
        if num_dims == 2:
            # 只有2维，说明是对全连接BN，[batch_size, n_features]
            shape = (1, num_features)
        else:
            # 是要对卷积BN，[B, C, H, W]
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成1和0
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 非模型参数的变量初始化为0和1
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var
        # 复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var
        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

In [75]:
net = nn.Sequential(
    # 输入 [256, 1, 28, 28]

    # 28 - 5 + 1 = 24 -> [256, 6, 24, 24]
    nn.Conv2d(1, 6, kernel_size=5),

    # [256, 6, 24, 24]
    BatchNorm(6, num_dims=4),

    nn.Sigmoid(),

    # (24 - 2 + 2) / 2 = 12 -> [256, 6, 12, 12]
    nn.AvgPool2d(kernel_size=2, stride=2),

    # 12  - 5 + 1 = 8 -> [256, 16, 8, 8]
    nn.Conv2d(6, 16, kernel_size=5),

    # [256, 16, 8, 8]
    BatchNorm(16, num_dims=4),

    nn.Sigmoid(),

    # (8 - 2 + 2) / 2 = 4 -> [256, 16, 4, 4]
    nn.AvgPool2d(kernel_size=2, stride=2),

    # [256, 16 * 4 * 4] -> [256, 256]
    nn.Flatten(),

    # [256, 120]
    nn.Linear(16*4*4, 120),

    # [256, 120]
    BatchNorm(120, num_dims=2),

    nn.Sigmoid(),

    # [120, 84]
    nn.Linear(120, 84),

    # [120, 84]
    BatchNorm(84, num_dims=2),

    nn.Sigmoid(),

    # [84, 10]
    nn.Linear(84, 10))

In [73]:
summary(net, input_size=(1,28,28), batch_size=256)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [256, 6, 24, 24]             156
         BatchNorm-2           [256, 6, 24, 24]               0
           Sigmoid-3           [256, 6, 24, 24]               0
         AvgPool2d-4           [256, 6, 12, 12]               0
            Conv2d-5            [256, 16, 8, 8]           2,416
         BatchNorm-6            [256, 16, 8, 8]               0
           Sigmoid-7            [256, 16, 8, 8]               0
         AvgPool2d-8            [256, 16, 4, 4]               0
           Flatten-9                 [256, 256]               0
           Linear-10                 [256, 120]          30,840
        BatchNorm-11                 [256, 120]               0
          Sigmoid-12                 [256, 120]               0
           Linear-13                  [256, 84]          10,164
        BatchNorm-14                  [

In [71]:
lr, num_epochs, batch_size = 1.0, 1, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
# for i in train_iter:
#     print(i[0].shape)
# d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

看从第一个批量规范化层中学到的拉伸参数gamma和偏移参数beta

In [77]:
net[1].gamma, net[1].beta

(Parameter containing:
 tensor([[[[1.]],
 
          [[1.]],
 
          [[1.]],
 
          [[1.]],
 
          [[1.]],
 
          [[1.]]]], requires_grad=True),
 Parameter containing:
 tensor([[[[0.]],
 
          [[0.]],
 
          [[0.]],
 
          [[0.]],
 
          [[0.]],
 
          [[0.]]]], requires_grad=True))

# ResNet

随着设计越来越深的网络，
深刻理解“新添加的层如何提升网络的性能”变得至关重要。

由于
只有当较复杂的函数类包含较小的函数类时，
我们才能确保提高它们的性能。

对于深度神经网络，
如果我们能把新添加的层训练成恒等映射（identity function）$f(x)=x$，
新模型将和原模型同样有效。
并且，由于新模型可能提出更优的解来拟合训练数据集，
因此，添加层似乎更容易降低训练误差。

因此，残差网络的核心思想是：

每个附加层都应该更容易地包含原始函数作为元素之一。

## 残差块
![残差块](imgs/残差块.png)

![残差块](imgs/残差块2.png)

In [78]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

class Residual(nn.Module):
    def __init__(self, input_channels, num_channels, use1x1conv=False, strides=1):
        """
        残差块
        :param input_channels:输入通道数
        :param num_channels:内层卷积输出通道
        :param use1x1conv:是否使用1x1卷积
        :param strides:步长
        """
        super(Residual, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels,num_channels, kernel_size=3, padding=1)
        if use1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels, kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self,X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += Y
        return F.relu(Y)


如图和上面代码，此代码生成两种类型的残差块，
一种使用了1x1卷积调整通道和分辨率，
另一种再应用ReLU非线性函数之前，将输入添加到输出。

下面检查输入和输出一致的情况

In [79]:
blk = Residual(input_channels=3, num_channels=3)
X = torch.rand(size=(4,3,6,6))
print(X.shape, blk(X).shape)

torch.Size([4, 3, 6, 6]) torch.Size([4, 3, 6, 6])


我们也可以在增加输出通道数的同时，减半输出的高和宽

In [82]:
blk = Residual(input_channels=3, num_channels=6, use1x1conv=True, strides=2)
print(blk(X).shape)

torch.Size([4, 6, 3, 3])


In [83]:
blk = Residual(input_channels=3, num_channels=6, use1x1conv=False, strides=2)
print(blk(X).shape)

torch.Size([4, 6, 3, 3])


## ResNet模型
![ResNet模型](imgs/resnet18.png)

In [84]:
# ResNet的前两层跟之前介绍的GoogLeNet中的一样：
# 在输出通道数为64、步幅为2的卷积层后，接步幅为2的的最大汇聚层。
# 不同之处在于ResNet每个卷积层后增加了批量规范化层。
b1 = nn.Sequential(
    # 输入1 * 224 * 224

    # （224 - 7 + 6 + 2） / 2 = 112
    nn.Conv2d(1,64, kernel_size=7, stride=2, padding=3),

    nn.BatchNorm2d(64),
    nn.ReLU(),

    # （112 - 3 + 2 + 2） / 2 = 56
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

In [85]:
summary(b1, input_size=(1,224, 224), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 112, 112]           3,200
       BatchNorm2d-2          [1, 64, 112, 112]             128
              ReLU-3          [1, 64, 112, 112]               0
         MaxPool2d-4            [1, 64, 56, 56]               0
Total params: 3,328
Trainable params: 3,328
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 19.91
Params size (MB): 0.01
Estimated Total Size (MB): 20.11
----------------------------------------------------------------


GoogLeNet在后面添加了4个由Inception组成的模块，
ResNet则使用4个由残差块组成的模块，

每个模块使用若干个相同输出通道的残差块。

第一个模块同输入通道一致，
由于之前已经使用了步幅为2的最大汇聚层，
所以无需减少高和宽。

之后每个模块在第一个模块的基础上把通道数翻倍，高和宽减半。

下面我们来实现这个模块，注意第一个模块做了特殊处理。

In [86]:
def resnet_block(input_channels, num_channels, num_residuals, first_block=False):
    """
    # 输入[batch_size, 64, 56, 56]
    :param input_channels:输入通道
    :param num_channels:输出通道
    :param num_residuals:组合几个残差块
    :param first_block:是否是一个模块
    :return:
    """
    blk = []
    for i in range(num_residuals):
        # 如果是每个resnet_block的第1块且不是b2
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels, use1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [87]:
# b2 没有添加1x1 卷积，也不改变通道和尺寸
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))

# b3 通道翻倍，尺寸减半
b3 = nn.Sequential(*resnet_block(64, 128, 2))

# b4 通道翻倍，尺寸减半
b4 = nn.Sequential(*resnet_block(128, 256, 2))

# b5 通道翻倍，尺寸减半
b5 = nn.Sequential(*resnet_block(256, 512, 2))

# 最后，与GoogLeNet一样，在ResNet中加入全局平均汇聚层，以及全连接层输出。
net = nn.Sequential(b1, b2, b3, b4, b5,
                    # [1, 512, 7, 7]

                    # [1, 512, 1, 1]
                    nn.AdaptiveAvgPool2d((1,1)),

                    # [1, 512]
                    nn.Flatten(),
                    # [512, 10]
                    nn.Linear(512, 10))

In [88]:
summary(b2, input_size=(64, 56, 56), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [1, 64, 56, 56]          36,928
       BatchNorm2d-2            [1, 64, 56, 56]             128
            Conv2d-3            [1, 64, 56, 56]          36,928
       BatchNorm2d-4            [1, 64, 56, 56]             128
          Residual-5            [1, 64, 56, 56]               0
            Conv2d-6            [1, 64, 56, 56]          36,928
       BatchNorm2d-7            [1, 64, 56, 56]             128
            Conv2d-8            [1, 64, 56, 56]          36,928
       BatchNorm2d-9            [1, 64, 56, 56]             128
         Residual-10            [1, 64, 56, 56]               0
Total params: 148,224
Trainable params: 148,224
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.77
Forward/backward pass size (MB): 15.31
Params size (MB): 0.57
Estimated 

In [90]:
summary(b3, input_size=(64, 56, 56), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [1, 128, 28, 28]          73,856
       BatchNorm2d-2           [1, 128, 28, 28]             256
            Conv2d-3           [1, 128, 28, 28]         147,584
       BatchNorm2d-4           [1, 128, 28, 28]             256
            Conv2d-5           [1, 128, 28, 28]           8,320
          Residual-6           [1, 128, 28, 28]               0
            Conv2d-7           [1, 128, 28, 28]         147,584
       BatchNorm2d-8           [1, 128, 28, 28]             256
            Conv2d-9           [1, 128, 28, 28]         147,584
      BatchNorm2d-10           [1, 128, 28, 28]             256
         Residual-11           [1, 128, 28, 28]               0
Total params: 525,952
Trainable params: 525,952
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.77
Forward/

In [91]:
summary(b4, input_size=(128, 28, 28), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [1, 256, 14, 14]         295,168
       BatchNorm2d-2           [1, 256, 14, 14]             512
            Conv2d-3           [1, 256, 14, 14]         590,080
       BatchNorm2d-4           [1, 256, 14, 14]             512
            Conv2d-5           [1, 256, 14, 14]          33,024
          Residual-6           [1, 256, 14, 14]               0
            Conv2d-7           [1, 256, 14, 14]         590,080
       BatchNorm2d-8           [1, 256, 14, 14]             512
            Conv2d-9           [1, 256, 14, 14]         590,080
      BatchNorm2d-10           [1, 256, 14, 14]             512
         Residual-11           [1, 256, 14, 14]               0
Total params: 2,100,480
Trainable params: 2,100,480
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.38
Forw

In [92]:
summary(b5, input_size=(256, 14, 14), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [1, 512, 7, 7]       1,180,160
       BatchNorm2d-2             [1, 512, 7, 7]           1,024
            Conv2d-3             [1, 512, 7, 7]       2,359,808
       BatchNorm2d-4             [1, 512, 7, 7]           1,024
            Conv2d-5             [1, 512, 7, 7]         131,584
          Residual-6             [1, 512, 7, 7]               0
            Conv2d-7             [1, 512, 7, 7]       2,359,808
       BatchNorm2d-8             [1, 512, 7, 7]           1,024
            Conv2d-9             [1, 512, 7, 7]       2,359,808
      BatchNorm2d-10             [1, 512, 7, 7]           1,024
         Residual-11             [1, 512, 7, 7]               0
Total params: 8,395,264
Trainable params: 8,395,264
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forw

In [95]:
tmp = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(),
                    nn.Linear(512, 10))
summary(tmp, input_size=(512, 7, 7), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
 AdaptiveAvgPool2d-1             [1, 512, 1, 1]               0
           Flatten-2                   [1, 512]               0
            Linear-3                    [1, 10]           5,130
Total params: 5,130
Trainable params: 5,130
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.10
Forward/backward pass size (MB): 0.01
Params size (MB): 0.02
Estimated Total Size (MB): 0.12
----------------------------------------------------------------


In [96]:
# lr, num_epochs, batch_size = 0.05, 10, 256
# train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
# d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

# 稠密连接网络

回想一下。任意函数的泰勒展开式，

它把函数分解成越来越高阶的项，在x->0时，

![泰勒展开](imgs/泰勒展开.png)

同样，把ResNet函数展开：
$$
f(x) = x + g(x)
$$

也就是说，ResNet把$f$分解成了两部分：

一个简单的线性项和一个复杂的非线性项。

那么再向前扩展一步，如果把$f$扩展成超过两部分的信息呢？

一种方案就是DenseNet。

![DenseAndRes](imgs/DenseNet与ResNet.png)

ResNet（左）与 DenseNet（右）在跨层连接上的主要区别：使用相加和使用连结。

ResNet与DenseNet的关键区别在于，
DenseNet的输出时连接，而不是ResNet的简单相加。

稠密网络主要由2部分构成：
稠密块（dense block）和过渡层（transition layer）。
前者定义如何连接输入和输出，后者控制通道数量，使其不会过于复杂。

## 稠密块体

In [99]:
from torch import nn
from d2l import torch as d2l
import torch

def conv_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels),
        nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1)
    )

一个稠密块由多个卷积块组成，

每个卷积块使用相同数量的输出通道。

然而，在前向传播中，

我们将每个卷积块的输入和输出在通道维度上连结。

In [154]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, input_channels, num_channels):
        """
        构建DenseBlock，每个DenseBlock由组合卷积构成，并通道数目增加
        :param num_convs: 使用几个组合卷积
        :param input_channels: 输入通道数
        :param num_channels: 输出通道增长率
        """
        super(DenseBlock, self).__init__()
        layer = []
        for i in range(num_convs):
            # print(f'{i}. conv_block({i * num_channels + input_channels, num_channels})')

            layer.append(
                conv_block(i * num_channels + input_channels, num_channels)
            )
        self.net = nn.Sequential(*layer)
    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            # 连接通道维度上每个块的输入和输出
            # print('X.shape', X.shape, 'Y.shape', Y.shape)
            X = torch.cat((X,Y),dim=1)
        return X

下面我们定义一个有2个输出通道为10的DenseBlock。

使用通道数为3的输入时，
我们将会得到通道数为3 + 2 * 10 = 23的输出。

卷积块的通道数控制了输出通道数相对于输入通道数的增长，

因此也被称为增长率（growth rate）

In [145]:
# 2个输出通道为10的DenseBlock，输入通道为3，10就是增长率
blk = DenseBlock(2, 3, 10)
X = torch.randn(4, 3, 8, 8)
Y = blk(X)
Y.shape

torch.Size([4, 23, 8, 8])

也就是，每次都经过conv_block都会把结果按通道拼接，送入下一个conv_block

## 过渡层
由于每个稠密块都会带来通道数的增加，
使用过多会产生过于复杂的模型。

过渡层可以控制模型的复杂度。

它使用1 x 1卷积来减少通道数，
并使用步幅为2的平均汇聚层来减半H和W，
从而进一步降低模型复杂度。

In [146]:
def transition_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels),
        nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=1),
        nn.AvgPool2d(kernel_size=2, stride=2)
    )

In [147]:
# 使用通道数为10的过渡层。 此时输出的通道数减为10，高和宽均减半。
blk = transition_block(23, 10)
blk(Y).shape

torch.Size([4, 10, 4, 4])

## DenseNet模型


In [148]:
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
# 输出[1, 64, 24, 24]

In [149]:
summary(b1, input_size=(1,96,96), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [1, 64, 48, 48]           3,200
       BatchNorm2d-2            [1, 64, 48, 48]             128
              ReLU-3            [1, 64, 48, 48]               0
         MaxPool2d-4            [1, 64, 24, 24]               0
Total params: 3,328
Trainable params: 3,328
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.04
Forward/backward pass size (MB): 3.66
Params size (MB): 0.01
Estimated Total Size (MB): 3.70
----------------------------------------------------------------


接下来，类似与ResNet使用4个残差块，DenseNet使用4个稠密块。

与ResNet类似，我们可以设置每个稠密块使用多少个卷积层。
这里我们设置4，以和resnet18保持一致。

稠密块里卷积层通道数（增长率）设置为32，所以每个稠密块将增长128个通道。

在每个模块之间，ResNet通过步幅为2的残差块减小高和宽，
DenseNet则使用过渡层来减半高和宽，并减半通道数。

In [150]:
# num_channels为当前的通道数
# # [1, 64, 24, 24]

num_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [4, 4, 4, 4]

blks = []
for i, num_convs in enumerate(num_convs_in_dense_blocks):
    # DenseBlock(4,64,32)
    blks.append(DenseBlock(num_convs, num_channels, growth_rate))
    # 更新上一个稠密块的输出通道数，便于下一轮循环继续构建
    num_channels += num_convs * growth_rate
    # 在稠密块之间增加一个过渡层
    if i != len(num_convs_in_dense_blocks)-1:
        blks.append(transition_block(num_channels, num_channels//2))
        num_channels = num_channels // 2

In [155]:
net = nn.Sequential(
    # 输入[1,1,96,96]

    # [1, 64, 24, 24]
    b1,
    # [1, 248, 3, 3]
    *blks,
    # [1, 248, 3, 3]
    nn.BatchNorm2d(num_channels), nn.ReLU(),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(num_channels, 10))

In [156]:
summary(net, input_size=(1,96,96), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [1, 64, 48, 48]           3,200
       BatchNorm2d-2            [1, 64, 48, 48]             128
              ReLU-3            [1, 64, 48, 48]               0
         MaxPool2d-4            [1, 64, 24, 24]               0
       BatchNorm2d-5            [1, 64, 24, 24]             128
              ReLU-6            [1, 64, 24, 24]               0
            Conv2d-7            [1, 32, 24, 24]          18,464
       BatchNorm2d-8            [1, 96, 24, 24]             192
              ReLU-9            [1, 96, 24, 24]               0
           Conv2d-10            [1, 32, 24, 24]          27,680
      BatchNorm2d-11           [1, 128, 24, 24]             256
             ReLU-12           [1, 128, 24, 24]               0
           Conv2d-13            [1, 32, 24, 24]          36,896
      BatchNorm2d-14           [1, 160,

In [157]:
# lr, num_epochs, batch_size = 0.1, 10, 256
# train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
# d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())